In [1]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)

import os, argparse, sys
# sys.path.append('./')
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
os.chdir(ROOT_DIR)
sys.path.append(ROOT_DIR)
# importing scripts in scripts folder
from scripts import config as src

**** scripts/config.py IMPORTED!!!
**** PROJECT FOLDER ,  /home/pgundogdu/projects/signalization_prior_knowledge_based_nn


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.model_selection import LeavePGroupsOut, train_test_split
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, fowlkes_mallows_score

epochs_default=100 # for model design
batch_default=10   # for model design
val_split=0.1      # for model design
rand_state = 91    # dataset split
test_size = 0.3    # train_test_split
kf_split = 5       # KFold

from numba import cuda
from tensorflow import keras


In [3]:
loc_output = 'models/NN/exper_mouse/leavePgroupsOut'
save_path = src.define_folder(loc_output)
save_path

FOLDER information,  models/NN/exper_mouse/leavePgroupsOut/


'models/NN/exper_mouse/leavePgroupsOut/'

In [4]:
df_learning = pd.read_csv('data/external/exper_mouse/1-3_integrated_NNtraining.txt'
                          , sep='\t'
                          , index_col=0).T.drop(columns='Weight')#.reset_index()
df_learning = df_learning.rename(columns={'Label':'cell_type'})
df_learning = df_learning[sorted(df_learning.columns[1:]) + [df_learning.columns[0] ]]
print(df_learning.shape)
df_learning.head()

(402, 9438)


Sample,a1bg,a1cf,a2m,aaas,aacs,aagab,aak1,aamp,aanat,aars,...,zswim7,zufsp,zw10,zwilch,zxdb,zxdc,zyx,zzef1,zzz3,cell_type
S1,0,0,0.0,103.853,0.0586557,0,3.97772,136.672,0,28.4994,...,0,0,0.278639,0,0.0,0.0,297.919,37.6855,0,BMDC
S2,0,0,0.0,0,0,0,0.00742256,440.725,0,3.15802,...,0,147.538,0,0,0.0,0.0,0.242888,0,0,BMDC
S3,0,0,0.0,0,0,0,0.0261848,118.676,0,0,...,0,17.7033,0.0863634,0,0.0,0.0,6.75353,0.132011,0.0176152,BMDC
S4,0,0,0.0,0,0,0,0,355.623,0,9.10864,...,0,0.537516,0.0636703,0,0.0,0.0,0.274689,9.72489,0,BMDC
S5,0,0,0.0,0,0,0,0,151.626,0,19.3458,...,0,128.928,0,0.204522,0.0,0.0,59.2561,44.4307,0,BMDC


In [14]:
scaler = MinMaxScaler()
df_learning.iloc[:, :-1] = scaler.fit_transform(df_learning.iloc[:, :-1])
print(df_learning.shape)
df_learning.head()

(402, 9438)


Sample,a1bg,a1cf,a2m,aaas,aacs,aagab,aak1,aamp,aanat,aars,...,zswim7,zufsp,zw10,zwilch,zxdb,zxdc,zyx,zzef1,zzz3,cell_type
S1,0,0,0,0.193008,0.000101553,0,0.0968245,0.10984,0,0.102434,...,0,0,0.00143474,0,0,0,0.413223,0.652245,0,BMDC
S2,0,0,0,0,0,0,0.000180678,0.354199,0,0.0113507,...,0,0.302217,0,0,0,0,0.000336894,0,0,BMDC
S3,0,0,0,0,0,0,0.000637382,0.0953768,0,0,...,0,0.0362633,0.000444694,0,0,0,0.00936737,0.00228479,9.9952e-05,BMDC
S4,0,0,0,0,0,0,0,0.285804,0,0.0327387,...,0,0.00110105,0.000327845,0,0,0,0.000381003,0.168314,0,BMDC
S5,0,0,0,0,0,0,0,0.121858,0,0.0695334,...,0,0.264095,0,0.000588339,0,0,0.0821901,0.768988,0,BMDC


In [15]:
df_scaled = df_learning.reset_index(drop=True)
print(df_scaled.shape)
df_scaled.head()

(402, 9438)


Sample,a1bg,a1cf,a2m,aaas,aacs,aagab,aak1,aamp,aanat,aars,...,zswim7,zufsp,zw10,zwilch,zxdb,zxdc,zyx,zzef1,zzz3,cell_type
0,0,0,0,0.193008,0.000101553,0,0.0968245,0.10984,0,0.102434,...,0,0,0.00143474,0,0,0,0.413223,0.652245,0,BMDC
1,0,0,0,0,0,0,0.000180678,0.354199,0,0.0113507,...,0,0.302217,0,0,0,0,0.000336894,0,0,BMDC
2,0,0,0,0,0,0,0.000637382,0.0953768,0,0,...,0,0.0362633,0.000444694,0,0,0,0.00936737,0.00228479,9.9952e-05,BMDC
3,0,0,0,0,0,0,0,0.285804,0,0.0327387,...,0,0.00110105,0.000327845,0,0,0,0.000381003,0.168314,0,BMDC
4,0,0,0,0,0,0,0,0.121858,0,0.0695334,...,0,0.264095,0,0.000588339,0,0,0.0821901,0.768988,0,BMDC


In [16]:
df_pathway_signaling = pd.read_csv('data/processed/pbk_layer_mmu.txt', index_col=0)
df_pathway_metsig = pd.read_csv('data/processed/pbk_layer_mmu_metsig.txt', index_col=0)
df_ppi = pd.read_csv('data/processed/pbk_layer_ppi.txt', index_col=0)
df_ppitf = pd.read_csv('data/processed/pbk_layer_ppitf.txt', index_col=0)

print(df_pathway_signaling.shape)
print(df_pathway_metsig.shape)
print(df_ppi.shape)
print(df_ppitf.shape)

df_scaled = pd.concat([df_scaled.iloc[:, :-1].astype(np.float32).reset_index(drop=True), df_scaled.iloc[:, -1:].reset_index(drop=True)], axis=1)
print(df_scaled.info())
df_scaled_with_pathway_signaling = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_pathway_signaling.index) )+ ['cell_type'] )]
df_scaled_with_pathway_metsig = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_pathway_metsig.index) )+ ['cell_type'] )]
df_scaled_with_ppi = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_ppi.index) )+ ['cell_type'] )]
df_scaled_with_ppitf = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_ppitf.index) )+ ['cell_type'] )]

print(df_scaled_with_pathway_signaling.shape)
print(df_scaled_with_pathway_metsig.shape)
print(df_scaled_with_ppi.shape)
print(df_scaled_with_ppitf.shape)

(3737, 92)
(3922, 250)
(3553, 348)
(9437, 696)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Columns: 9438 entries, a1bg to cell_type
dtypes: float32(9437), object(1)
memory usage: 14.5+ MB
None
(402, 1646)
(402, 3923)
(402, 3554)
(402, 9438)


In [17]:
callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss" # Stop training when `val_loss` is no longer improving
                                       , min_delta=1e-5   # "no longer improving" being defined as "no better than 1e-5 less"
                                       , patience=3       # "no longer improving" being further defined as "for at least 3 epochs"
                                       , verbose=1 ) ]

In [18]:
n_groups_ = [2,4,6,8]
ohe = OneHotEncoder()
# n_groups_ = [4]

X = df_scaled.iloc[:, :-1].values
y = df_scaled.iloc[:, -1:].values
y_ohe = ohe.fit_transform(y).toarray()
groups = df_scaled.iloc[:, -1].values


In [1]:
# FULLY CONNECTED 1-LAYER-100
# df_scaler_use = df_scaled.copy()
# df_bio_use = None
# second_layer=False
# size_units = 100
# design_type='fully'
# design = 'full_gene_space_1_layer_dense100'

# FULLY CONNECTED 1-LAYER-796
# df_scaler_use = df_scaled.copy()
# df_bio_use = None
# second_layer=False
# size_units = 796
# design_type='fully'
# design = 'full_gene_space_1_layer_dense796'

In [2]:
# # FULLY CONNECTED 1-LAYER-100 with signaling gene space
# df_scaler_use = df_scaled_with_pathway_signaling.copy()
# df_bio_use = None
# second_layer=False
# size_units = 100
# design_type='fully'
# design = 'signaling_gene_space_1_layer_dense100'

# # # FULLY CONNECTED 1-LAYER-796 with signaling gene space
# df_scaler_use = df_scaled_with_pathway_signaling.copy()
# df_bio_use = None
# second_layer=False
# size_units = 796
# design_type='fully'
# design = 'signaling_gene_space_1_layer_dense796'

In [3]:
# # FULLY CONNECTED 1-LAYER-100 with metabolic signaling gene space
# df_scaler_use = df_scaled_with_pathway_metsig.copy()
# df_bio_use = None
# second_layer=False
# size_units = 100
# design_type='fully'
# design = 'metsig_gene_space_1_layer_dense100'

# # FULLY CONNECTED 1-LAYER-796 with metabolic signaling gene space
# df_scaler_use = df_scaled_with_pathway_metsig.copy()
# df_bio_use = None
# second_layer=False
# size_units = 796
# design_type='fully'
# design = 'metsig_gene_space_1_layer_dense796'

In [10]:
# # SIGNALING 1-LAYER
# df_scaler_use = df_scaled_with_pathway_signaling.copy()
# df_bio_use = df_pathway_signaling.copy()
# second_layer=False
# size_units = 100
# design_type='bio'
# design = 'signaling_gene_space_1_layer_pathway'

# # SIGNALING 2-LAYER
# df_scaler_use = df_scaled_with_pathway_signaling.copy()
# df_bio_use = df_pathway_signaling.copy()
# second_layer=True
# size_units = 100
# design_type='bio'
# design = 'signaling_gene_space_2_layer_pathway'

In [10]:
# # METABOLIC SIGNALING 1-LAYER
# df_scaler_use = df_scaled_with_pathway_metsig.copy()
# df_bio_use = df_pathway_metsig.copy()
# second_layer=False
# size_units = 100
# design_type='bio'
# design = 'metsig_gene_space_1_layer_pathway'

# # METABOLIC SIGNALING 2-LAYER
# df_scaler_use = df_scaled_with_pathway_metsig.copy()
# df_bio_use = df_pathway_metsig.copy()
# second_layer=True
# size_units = 100
# design_type='bio'
# design= 'metsig_gene_space_2_layer_pathway'

In [19]:
df_dense = pd.DataFrame(df_scaled.columns[:-1]).set_index('Sample')
for i in range(100):
    df_dense['dense'+str(i)] = 1.0
df_first_hidden_layer = df_dense.copy()
df_first_hidden_layer

# df_first_hidden_layer = pd.merge(df_dense, df_ppi, left_index=True, right_index=True, how='left').fillna(0)
df_first_hidden_layer = pd.merge(df_dense, df_ppitf, left_index=True, right_index=True, how='left').fillna(0)

df_first_hidden_layer

,dense0,dense1,dense2,dense3,dense4,dense5,dense6,dense7,dense8,dense9,...,ppi_338,ppi_339,ppi_340,ppi_341,ppi_342,ppi_343,ppi_344,ppi_345,ppi_346,ppi_347
Sample,,,,,,,,,,,,,,,,,,,,,
a1bg,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a1cf,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a2m,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaas,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aacs,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zxdb,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zxdc,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zyx,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# FULLY CONNECTED 1-LAYER-100+PPİ
df_scaler_use = df_scaled.copy()
df_bio_use = df_first_hidden_layer
second_layer=False
size_units = 100
design_type='bio'
design = 'full_gene_space_1_layer_dense100+ppitf'

In [22]:
seed(1)
df_result = pd.DataFrame()
for i_pgroups in n_groups_[1:2]:
    
    print(i_pgroups)
    splitter = LeavePGroupsOut(n_groups=i_pgroups)
    splits = list(splitter.split(X, y, groups))
    ids = np.random.choice(len(splits), 20).tolist()
    list_random_selected = [splits[i] for i in ids]
    
    design_name = design+'-LeavePGroupOut_'+str(i_pgroups)

    # ## LEVAEPGROUPSOUT CONTROL
    # for i in range(len(list_random_selected)):
    #     print('EXPERIMENT -', str(i+1) , '/', str(len(list_random_selected)))
    #     print('kept            - ', list(df_scaled[df_scaled.index.isin(list_random_selected[i][0])].groupby('cell_type').size().index))
    #     print('LeavePGroupsOut - ', list(df_scaled[df_scaled.index.isin(list_random_selected[i][1])].groupby('cell_type').size().index))

    df_all_per_cell_out = pd.DataFrame()
    for i in range(len(list_random_selected)):
        print('EXPERIMENT -- ', str(i+1), '/', str(len(list_random_selected)))
        ohe_kept = OneHotEncoder()
        X_train, y_train, X_test, y_test = [], [], [], []

        df_kept_groups = df_scaler_use[df_scaler_use.index.isin(list_random_selected[i][0])]
        if design_type == 'bio':
            df_first_hidden_layer = df_bio_use[df_bio_use.index.isin(list(df_kept_groups.columns))]
        else:
            df_first_hidden_layer = pd.DataFrame()

        X_kept = df_kept_groups.iloc[:, :-1].values
        y_kept = df_kept_groups.iloc[:, -1:].values#y_ohe[list_random_selected[i][0]]
        y_ohe_kept = ohe_kept.fit_transform(y_kept).toarray()

        print(X_kept.shape)
        print(y_kept.shape)
        print(y_ohe_kept.shape)


        Xtrain, Xtest, ytrain, ytest = train_test_split(X_kept, y_kept
                                                        , test_size=test_size
                                                        , shuffle=True
                                                        , random_state=rand_state
                                                        , stratify=y_ohe_kept
                                                       )

        X_train.append(Xtrain)
        X_test.append(Xtest)
        y_train.append(ohe_kept.transform(ytrain).toarray())
        y_test.append(ohe_kept.transform(ytest).toarray())


        model_trained = src.proposed_NN(X=X_kept
                                        , y=y_kept
                                        , bio_layer=df_first_hidden_layer
                                        , design_type=design_type
                                        , size_units=size_units
                                        , select_optimizer='SGD'
                                        , second_layer=second_layer)
        model_trained.fit(X_train[0], y_train[0]
                  , epochs=epochs_default
                  , batch_size=batch_default
                  , verbose=1
                  , callbacks=callbacks
                  , validation_split=val_split)
        
        model_trained.save(src.define_folder(os.path.join(save_path, 'cell_out_'+str(i_pgroups))) + design+'_'+str(i)+'.h5')
        
        y_pred = model_trained.predict(X_test[0])
        df_proba = pd.DataFrame(y_pred, columns=list(pd.DataFrame(ohe_kept.categories_).iloc[0,:]))
        df_pred = pd.DataFrame(ohe_kept.inverse_transform(y_pred).reshape(1,-1)[0], columns=['prediction'])
        df_ground_truth = pd.DataFrame(ohe_kept.inverse_transform(np.array(y_test[0])).reshape(1,-1)[0], columns=['ground_truth'])
        df_nn = pd.concat([df_proba, df_pred, df_ground_truth], axis=1)
        df_nn['experiment_index'] = i+1
        df_all_per_cell_out = pd.concat([df_all_per_cell_out , df_nn], axis=0)
    

    list_homo, list_comp, list_vmes, list_ari, list_ami, list_fm, list_acc = [],[],[],[],[],[],[]

    for i_exp in df_all_per_cell_out['experiment_index'].unique():
        df_temp = df_all_per_cell_out[(df_all_per_cell_out['experiment_index']==i_exp)]

        list_homo.append([ design_name, homogeneity_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'homogeneity'])
        list_comp.append([ design_name, completeness_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'completeness'])
        list_vmes.append([ design_name, v_measure_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'v_measure'])
        list_ari.append([ design_name, adjusted_rand_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'ari'])
        list_ami.append([ design_name, adjusted_mutual_info_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'ami'])
        list_fm.append([ design_name, fowlkes_mallows_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'fowlkes_mallows'])

    result = [element for lis in [list_homo, list_comp, list_vmes, list_ari, list_ami, list_fm] for element in lis]

    df_metric = pd.DataFrame(result, columns=['experiment','score','expr','metric'])


    df_result = pd.concat([df_result 
                           , pd.DataFrame(df_metric.groupby(['experiment', 'metric']).mean()['score']).pivot_table(index='experiment', columns='metric')])
    
df_result.to_csv(os.path.join(save_path, design+'.csv'))

4
EXPERIMENT --  1 / 20
(325, 9437)
(325, 1)
(325, 12)
  -> Network designed with prior biological knowledge with 796 nodes in first hidden layer.
    -> do not need to define size_units!!
------------- NETWORK DESIGN - ARGUMENTS -------------
-- X.shape                  , (325, 9437)
-- y.shape                  , (325, 1)
-- bio_layer.shape          , (9437, 796)
-- design_type              , bio
------------- NETWORK DESIGN - CALCULATED -------------
-- input_size               , 9437
-- first_hidden_layer_size  , 796
-- size_output_layer        , 12


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
first_hidden_layer_size 796
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)

KeyboardInterrupt: 